# LSTM Autoencoder Hyperparameter Tuning

This notebook performs hyperparameter optimization for LSTM Autoencoder using Optuna.

**Task**: Binary anomaly detection (normal vs fault)

**Architecture**: LSTM encoder-decoder for reconstruction-based anomaly detection. Trained on normal data only, anomalies are detected by high reconstruction error.

**Data Handling**:
- Windows are created within simulation runs only (no cross-run windows)
- Subsampling is done by simulation runs, not individual rows

**Outputs**:
- Best hyperparameters: `outputs/hyperparams/lstm_autoencoder_best.json`
- Optuna study: `outputs/optuna_studies/lstm_autoencoder_study.pkl`

In [1]:
import os
import sys
import time
import json
import pickle
from pathlib import Path

start_time = time.time()
print("="*60)
print("LSTM Autoencoder Hyperparameter Tuning")
print("="*60)
print(f"Started at: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))}")

QUICK_MODE = os.getenv('QUICK_MODE', 'False').lower() in ('true', '1', 'yes')

if QUICK_MODE:
    RUN_FRACTION = 0.01
    MIN_RUNS_PER_CLASS = 5
    N_TRIALS = 5
    MAX_EPOCHS = 10
    PATIENCE = 3
    print("🚀 QUICK MODE (1% runs, min 5/class, 5 trials, max 10 epochs)")
else:
    RUN_FRACTION = 0.50
    MIN_RUNS_PER_CLASS = 5
    N_TRIALS = 50
    MAX_EPOCHS = 50
    PATIENCE = 5
    print("🔬 TUNING MODE (50% runs, 50 trials, max 50 epochs)")

DATA_DIR = Path('../data')
OUTPUT_DIR = Path('../outputs')
HYPERPARAM_DIR = OUTPUT_DIR / 'hyperparams'
STUDY_DIR = OUTPUT_DIR / 'optuna_studies'
PROGRESS_FILE = OUTPUT_DIR / 'lstm_autoencoder_progress.log'

HYPERPARAM_DIR.mkdir(parents=True, exist_ok=True)
STUDY_DIR.mkdir(parents=True, exist_ok=True)

RANDOM_SEED = 42

print(f"Run fraction: {RUN_FRACTION*100}%")
print(f"Trials: {N_TRIALS}, Max epochs: {MAX_EPOCHS}, Patience: {PATIENCE}")
print("="*60)

def log_progress(message):
    print(message, flush=True)
    with open(PROGRESS_FILE, 'a') as f:
        f.write(f"{message}\n")
        f.flush()

PROGRESS_FILE.write_text("")

LSTM Autoencoder Hyperparameter Tuning
Started at: 2026-01-04 16:15:31
🔬 TUNING MODE (50% runs, 50 trials, max 50 epochs)
Run fraction: 50.0%
Trials: 50, Max epochs: 50, Patience: 5


0

In [2]:
log_progress("\n[Step 1/6] Loading libraries...")
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
import optuna
from optuna.pruners import MedianPruner
import warnings
warnings.filterwarnings('ignore')

torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
log_progress(f"✓ Using device: {device}")


[Step 1/6] Loading libraries...


✓ Using device: cuda


In [3]:
log_progress("\n[Step 2/6] Loading data...")

train = pd.read_csv(DATA_DIR / 'binary_train.csv')
val = pd.read_csv(DATA_DIR / 'binary_val.csv')
test = pd.read_csv(DATA_DIR / 'binary_test.csv')

log_progress(f"✓ Full data - Train: {train.shape}, Val: {val.shape}, Test: {test.shape}")

def sample_by_runs(df, fraction, seed, min_runs=5):
    """Sample complete simulation runs, preserving temporal structure."""
    runs = df.groupby(['faultNumber', 'simulationRun']).size().reset_index()[['faultNumber', 'simulationRun']]
    
    def sample_class(x):
        n_total = len(x)
        n_sample = max(min_runs, int(n_total * fraction))
        n_sample = min(n_sample, n_total)
        return x.sample(n=n_sample, random_state=seed)
    
    sampled_runs = runs.groupby('faultNumber', group_keys=False).apply(sample_class)
    df_sampled = df.merge(sampled_runs, on=['faultNumber', 'simulationRun'])
    return df_sampled.sort_values(['faultNumber', 'simulationRun', 'sample']).reset_index(drop=True)

# Sample training data - we only use normal data for training
train_sampled = sample_by_runs(train, RUN_FRACTION, RANDOM_SEED, MIN_RUNS_PER_CLASS)
train_normal = train_sampled[train_sampled['faultNumber'] == 0]

# For evaluation, we need BOTH normal and fault samples
# The binary dataset has: train/val = normal only, test = normal + faults
# So we use the TEST set for hyperparameter evaluation
test_sampled = sample_by_runs(test, RUN_FRACTION, RANDOM_SEED, MIN_RUNS_PER_CLASS)

n_train_runs = train_normal.groupby(['faultNumber', 'simulationRun']).ngroups
n_test_runs = test_sampled.groupby(['faultNumber', 'simulationRun']).ngroups

# Create binary label for evaluation (0=normal, 1=fault)
test_sampled['label'] = (test_sampled['faultNumber'] != 0).astype(int)

# Check class distribution in test set
n_normal = (test_sampled['label'] == 0).sum()
n_fault = (test_sampled['label'] == 1).sum()

log_progress(f"✓ Train (normal only): {train_normal.shape} ({n_train_runs} runs)")
log_progress(f"✓ Eval (from test): {test_sampled.shape} ({n_test_runs} runs)")
log_progress(f"  - Normal samples: {n_normal}, Fault samples: {n_fault}")
log_progress(f"  - Fault rate: {test_sampled['label'].mean():.2%}")

features = [col for col in train.columns if 'xmeas' in col or 'xmv' in col]
num_features = len(features)

scaler = StandardScaler()
scaler.fit(train_normal[features])

log_progress(f"✓ Features: {num_features}")


[Step 2/6] Loading data...


✓ Full data - Train: (50000, 57), Val: (25000, 57), Test: (795200, 57)


✓ Train (normal only): (25000, 57) (50 runs)


✓ Eval (from test): (397600, 58) (485 runs)


  - Normal samples: 57600, Fault samples: 340000


  - Fault rate: 85.51%


✓ Features: 52


In [4]:
log_progress("\n[Step 3/6] Defining LSTM Autoencoder model...")

class SimulationRunDataset(Dataset):
    """Dataset for autoencoder - creates windows WITHIN simulation runs only."""
    def __init__(self, df, features, scaler, sequence_length=10):
        self.seq_len = sequence_length
        self.windows = []
        
        for (fault, run), group in df.groupby(['faultNumber', 'simulationRun']):
            group = group.sort_values('sample')
            X = scaler.transform(group[features].values)
            
            for i in range(len(X) - sequence_length + 1):
                self.windows.append(X[i:i+sequence_length])
        
        self.windows = np.array(self.windows, dtype=np.float32)
    
    def __len__(self):
        return len(self.windows)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.windows[idx])

class LabeledSimulationRunDataset(Dataset):
    """Dataset with labels for evaluation."""
    def __init__(self, df, features, scaler, sequence_length=10):
        self.seq_len = sequence_length
        self.windows = []
        self.labels = []
        
        for (fault, run), group in df.groupby(['faultNumber', 'simulationRun']):
            group = group.sort_values('sample')
            X = scaler.transform(group[features].values)
            # label column created in load-data cell: 0=normal, 1=fault
            y = group['label'].values
            
            for i in range(len(X) - sequence_length + 1):
                self.windows.append(X[i:i+sequence_length])
                self.labels.append(y[i+sequence_length-1])
        
        self.windows = np.array(self.windows, dtype=np.float32)
        self.labels = np.array(self.labels, dtype=np.int64)
    
    def __len__(self):
        return len(self.windows)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.windows[idx]), torch.tensor(self.labels[idx])

class LSTMAutoencoder(nn.Module):
    """LSTM Autoencoder for anomaly detection."""
    def __init__(self, input_size, hidden_size, num_layers, latent_size, dropout=0.0):
        super().__init__()
        
        self.encoder = nn.LSTM(
            input_size, hidden_size, num_layers,
            batch_first=True, dropout=dropout if num_layers > 1 else 0
        )
        self.encoder_fc = nn.Linear(hidden_size, latent_size)
        
        self.decoder_fc = nn.Linear(latent_size, hidden_size)
        self.decoder = nn.LSTM(
            hidden_size, hidden_size, num_layers,
            batch_first=True, dropout=dropout if num_layers > 1 else 0
        )
        self.output_fc = nn.Linear(hidden_size, input_size)
        
    def forward(self, x):
        _, (h, _) = self.encoder(x)
        latent = self.encoder_fc(h[-1])
        
        h_dec = self.decoder_fc(latent)
        h_dec = h_dec.unsqueeze(1).repeat(1, x.size(1), 1)
        dec_out, _ = self.decoder(h_dec)
        reconstruction = self.output_fc(dec_out)
        
        return reconstruction

log_progress("✓ LSTM Autoencoder model defined")


[Step 3/6] Defining LSTM Autoencoder model...


✓ LSTM Autoencoder model defined


In [5]:
log_progress("\n[Step 4/6] Setting up optimization...")

def objective(trial):
    # Sequence length range [20, 40] to include manuscript's default of 30
    sequence_length = trial.suggest_int('sequence_length', 20, 40)
    hidden_size = trial.suggest_categorical('hidden_size', [32, 64, 128])
    num_layers = trial.suggest_int('num_layers', 1, 2)
    latent_size = trial.suggest_categorical('latent_size', [8, 16, 32])
    dropout = trial.suggest_float('dropout', 0.0, 0.4)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    threshold_percentile = trial.suggest_float('threshold_percentile', 90.0, 99.0)
    
    train_dataset = SimulationRunDataset(train_normal, features, scaler, sequence_length)
    eval_dataset = LabeledSimulationRunDataset(test_sampled, features, scaler, sequence_length)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    eval_loader = DataLoader(eval_dataset, batch_size=batch_size)
    
    model = LSTMAutoencoder(
        num_features, hidden_size, num_layers, latent_size, dropout
    ).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Early stopping variables
    best_train_loss = float('inf')
    patience_counter = 0
    best_model_state = None
    
    for epoch in range(MAX_EPOCHS):
        # Training phase
        model.train()
        train_loss = 0.0
        for X_batch in train_loader:
            X_batch = X_batch.to(device)
            optimizer.zero_grad()
            reconstruction = model(X_batch)
            loss = criterion(reconstruction, X_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)
        
        # Early stopping based on training loss (autoencoders train on normal data only)
        if train_loss < best_train_loss:
            best_train_loss = train_loss
            patience_counter = 0
            best_model_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                break
    
    # Restore best model
    if best_model_state is not None:
        model.load_state_dict({k: v.to(device) for k, v in best_model_state.items()})
    
    model.eval()
    train_errors = []
    with torch.no_grad():
        for X_batch in train_loader:
            X_batch = X_batch.to(device)
            reconstruction = model(X_batch)
            errors = ((reconstruction - X_batch) ** 2).mean(dim=(1, 2))
            train_errors.extend(errors.cpu().numpy())
    
    threshold = np.percentile(train_errors, threshold_percentile)
    
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X_batch, y_batch in eval_loader:
            X_batch = X_batch.to(device)
            reconstruction = model(X_batch)
            errors = ((reconstruction - X_batch) ** 2).mean(dim=(1, 2))
            preds = (errors.cpu().numpy() > threshold).astype(int)
            all_preds.extend(preds)
            all_labels.extend(y_batch.numpy())
    
    return f1_score(all_labels, all_preds, average='weighted')

log_progress("✓ Objective function defined")


[Step 4/6] Setting up optimization...


✓ Objective function defined


In [6]:
log_progress(f"\n{'='*60}")
log_progress(f"[Step 5/6] Starting optimization")
log_progress(f"{'='*60}\n")

optuna_start = time.time()
optuna.logging.set_verbosity(optuna.logging.WARNING)

study = optuna.create_study(
    direction='maximize',
    pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=5),
    study_name='lstm_autoencoder_binary'
)

log_progress(f"Running {N_TRIALS} trials...")

for trial_num in range(N_TRIALS):
    study.optimize(objective, n_trials=1, show_progress_bar=False)
    trial = study.trials[-1]
    log_progress(f"Trial {trial_num + 1}/{N_TRIALS}: F1={trial.value:.4f} (best={study.best_value:.4f})")

optuna_time = time.time() - optuna_start

log_progress(f"\n{'='*60}")
log_progress("✓ Optimization complete!")
log_progress(f"Total time: {optuna_time:.2f}s")

[Step 5/6] Starting optimization


Running 50 trials...


Trial 1/50: F1=0.9481 (best=0.9481)


Trial 2/50: F1=0.9372 (best=0.9481)


Trial 3/50: F1=0.8235 (best=0.9481)


...

Trial 48/50: F1=0.9400 (best=0.9817)


Trial 49/50: F1=0.9768 (best=0.9817)


Trial 50/50: F1=0.9629 (best=0.9817)


✓ Optimization complete!


Total time: 9221.73s


In [7]:
end_time = time.time()
total_runtime = end_time - start_time

log_progress("\n[Step 6/6] Saving results...")

results = {
    'model': 'LSTM-Autoencoder',
    'task': 'binary',
    'best_params': study.best_params,
    'best_f1_weighted': float(study.best_value),
    'num_trials': N_TRIALS,
    'run_fraction': RUN_FRACTION,
    'quick_mode': QUICK_MODE,
    'max_epochs': MAX_EPOCHS,
    'early_stopping_patience': PATIENCE,
    'optimization_time_seconds': optuna_time,
    'random_seed': RANDOM_SEED,
    'timing': {
        'start_time': time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time)),
        'end_time': time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(end_time)),
        'total_runtime_seconds': float(total_runtime),
        'total_runtime_formatted': f"{int(total_runtime // 60)}m {int(total_runtime % 60)}s"
    }
}

mode_suffix = "_quick" if QUICK_MODE else ""
json_path = HYPERPARAM_DIR / f'lstm_autoencoder_best{mode_suffix}.json'
study_path = STUDY_DIR / f'lstm_autoencoder_study{mode_suffix}.pkl'

with open(json_path, 'w') as f:
    json.dump(results, f, indent=2)
log_progress(f"✓ Saved to {json_path}")

with open(study_path, 'wb') as f:
    pickle.dump(study, f)

log_progress(f"\n{'='*60}")
log_progress("✓ LSTM Autoencoder Hyperparameter Tuning Complete!")
log_progress(f"Runtime: {results['timing']['total_runtime_formatted']}")
log_progress(f"Best F1: {study.best_value:.4f}")
log_progress(f"{'='*60}")


[Step 6/6] Saving results...


✓ Saved to ../outputs/hyperparams/lstm_autoencoder_best.json


✓ LSTM Autoencoder Hyperparameter Tuning Complete!


Runtime: 153m 47s


Best F1: 0.9817
